# FILE where original DATA is analyzed and Transformed 


Our data is an extract of what we might find in our Google Analytics Data Stored in our Database

## Analysis of the Data 

In [16]:
import pandas as pd

df = pd.read_csv("google data/data.csv")
clicks_df = df.loc[df.Click == True]
buy_df = pd.read_csv("google data/purchases.csv")

In [3]:
clicks_df.head()  # df where is located user liking behavior linked to clicked products

,visitNumber,visitId,visitStartTime,date,total_visits,total_hits,total_pageviews,total_timeOnSite,total_bounces,total_newVisits,...,Category,Variant,ProductList,transactionId,Revenue,itemTransactionId,Click,TotalPaid,Price,Quantity
0,2,1470857552,1470857552,20160810,1,5,4,125.0,NaN,NaN,...,Home/Limited Supply/Bags/,(not set),Category,NaN,0.0,NaN,True,0.0,99.99,NaN
1,1,1470862136,1470862136,20160810,1,5,4,341.0,NaN,1.0,...,Home/Limited Supply/Bags/,(not set),Category,NaN,0.0,NaN,True,0.0,8.79,NaN
2,1,1470848713,1470848713,20160810,1,6,5,106.0,NaN,1.0,...,Home/Apparel/Men's/Men's-T-Shirts/,(not set),Category,NaN,0.0,NaN,True,0.0,16.99,NaN
3,4,1470815734,1470815734,20160810,1,6,3,8.0,NaN,NaN,...,Home/Office/,(not set),Category,NaN,0.0,NaN,True,0.0,0.99,NaN
4,4,1470815734,1470815734,20160810,1,6,3,8.0,NaN,NaN,...,Home/Office/,(not set),Category,NaN,0.0,NaN,True,0.0,0.99,NaN


In [4]:
buy_df.head()

,visitNumber,visitId,visitStartTime,date,total_visits,total_hits,total_pageviews,total_timeOnSite,total_bounces,total_newVisits,...,itemTransactionId,ProductName,ProductSKU,Category,Variant,ProductList,Revenue,TotalCost,Price,Quantity
0,2,1472571985,1472571985,20160830,1,13,11,305,NaN,NaN,...,ORD201608301467,24 oz YouTube Sergeant Stripe Bottle,GGOEYDHJ019399,(not set),Single Option Only,(not set),20.91,7.99,7.99,1
1,1,1472606991,1472606991,20160830,1,13,12,392,NaN,1.0,...,ORD201608301495,Google Laptop and Cell Phone Stickers,GGOEGFKQ020399,(not set),Single Option Only,(not set),112.00,99.50,1.99,50
2,3,1472599227,1472599227,20160830,1,14,12,192,NaN,NaN,...,ORD201608301510,20 oz Stainless Steel Insulated Tumbler,GGOEGDHQ014899,(not set),Single Option Only,(not set),11.24,19.99,19.99,1
3,1,1472594459,1472594459,20160830,1,16,14,685,NaN,1.0,...,ORD201608301546,Google Stretch Fit Hat M/L Navy,GGOEGHPL003214,(not set),M/L,(not set),29.49,21.99,21.99,1
4,3,1472604143,1472604143,20160830,1,16,13,895,NaN,NaN,...,ORD201608301559,Google 17oz Stainless Steel Sport Bottle,GGOEGDHC074099,(not set),Single Option Only,(not set),33.89,18.99,18.99,1


As we can see, the dataframe contains a description of all sifferent features that we can use for creating a valid analysis and further (the recommendation system). The most important part is that this data is obtained from a Google's BigQuery use case in this [Kaggle](https://www.kaggle.com/code/paultimothymooney/how-to-query-the-google-analytics-sample-dataset/notebook?scriptVersionId=5165120)

The most important features were selected from a list of features. I used a query to retrieve the products that were clicked by the users, as well as the confirmed purchases. This, along side some behavioral features like the time on which the users entered the site, the page views, time on site and other, all this can allow us to create behavior clusters in which we can create a user classification system and later classify behaviors into this already formed groups.



In [5]:
buy_df.drop_duplicates(
    subset="transactionId", keep="first", inplace=True
)  # Drop duplicate orders

In [6]:
buy_df["ProductName"].value_counts()  # Products that have being purchased before

Google Men's 100% Cotton Short Sleeve Hero Tee Navy     30
Google Men's  Zip Hoodie                                30
Google Men's 100% Cotton Short Sleeve Hero Tee Black    27
Google Men's 100% Cotton Short Sleeve Hero Tee White    25
Engraved Ceramic Google Mug                             23
                                                        ..
Google Executive Umbrella                                1
Google Accent Insulated Stainless Steel Bottle           1
Gel Roller Pen                                           1
Google Men's Performance Tee Gunmetal                    1
Bic Digital Clic Stic Stylus Pen                         1
Name: ProductName, Length: 248, dtype: int64

In [7]:
clicks_df["ProductName"].value_counts()  # Products that have being clicked

Mistral Rucksack                                          633
Deluge Waterproof Backpack                                575
Oasis Backpack                                            553
Keyboard DOT Sticker                                      552
Google Men's 100% Cotton Short Sleeve Hero Tee White      466
                                                         ... 
Google Toddler Tee Fruit Games Cherries                     3
Google Men's Short Sleeve Performance Badge Tee Pewter      2
Gift Card - $250.00                                         1
Google Toddler Tee Fruit Games Lemon                        1
Google Youth Tee Fruit Games Pineapple                      1
Name: ProductName, Length: 266, dtype: int64

We have to see how variables like Hits and Time on Site are distributed.

In [8]:
features_list = [
    "visitNumber",
    "total_visits",
    "total_hits",
    "Price",
    "total_timeOnSite",
    "total_newVisits",
    "total_pageviews",
]


clicks_df.loc[:, features_list].describe().round(
    2
)  # This would be the behavioral features that we have available

,visitNumber,total_visits,total_hits,Price,total_timeOnSite,total_newVisits,total_pageviews
count,20878.00,20878.0,20878.00,20878.00,20863.00,11736.0,20878.00
mean,4.22,1.0,50.96,28.11,923.26,1.0,33.45
std,13.46,0.0,51.92,30.15,1060.52,0.0,32.42
min,1.00,1.0,2.00,0.79,1.00,1.0,1.00
25%,1.00,1.0,17.00,5.99,243.00,1.0,12.00
50%,1.00,1.0,32.00,16.99,547.00,1.0,22.00
75%,3.00,1.0,65.00,34.99,1210.00,1.0,43.00
max,156.00,1.0,301.00,250.00,7508.00,1.0,188.00


## Embeddings 



Our system will work with what is called embedding a text in NLP. We take our text, map it into tokens, which are basically small id's for the word. Sometimes words are divided in multiple tokens. Which account for the fact that some words that are very similar or share a common root can be represented similarly. Those tokens are unique id's to the realm of what will be our embeddings realm. Which is nothing more than a vectorized space in which this tokens are represented in a numeric form --> $ \text{token:} i \rightarrow \vec{w_i} = <0.423,3.141,.674, ..., n>$.

When we have a sentence or a paragraph, embeddings are usually the average of all the tokens that we find in that structure, and there we have it, our embedding system. Usually this embeddings spaces are then trained on a corpus of text to represent different features. This is similar to what our brain does with sentences: we remember the nouns, the verbs, the grammar rules, and other complex rules that make our understanding of the language as it is.

On this demo, we use ***multilingual-e5-base*** model for creating a 768 size vector that will account for the features in our product title, which is the main thing we want to use as a driver of recommendations.


In [9]:
import sys

sys.path.append("..")

from main.models import ModelClass

model = ModelClass()

In [8]:
model.create_embeddings("Hello world")  # Use method create embeddings to use it.

array([[ 3.35431285e-02,  4.46086712e-02, -4.48141305e-04,
        -3.62625433e-04,  2.88973264e-02, -3.31031904e-02,
        -1.90947279e-02, -3.67002678e-03,  3.32840942e-02,
         3.20200399e-02, -2.73499619e-02, -1.29014198e-02,
         1.63098887e-01,  3.99894156e-02, -3.45210619e-02,
        -2.77194325e-02,  1.10267121e-02, -2.44755037e-02,
         2.67019086e-02, -1.29492851e-02,  5.62458374e-02,
        -2.56671403e-02,  3.23554426e-02, -3.01293898e-02,
         2.17710696e-02, -9.27064288e-03,  2.39941813e-02,
         1.11178188e-02, -3.06521188e-02,  3.21589038e-02,
         9.67866369e-03, -7.68294372e-03,  3.73637117e-02,
         1.61362011e-02,  3.65321301e-02,  4.55307253e-02,
        -8.69155489e-03, -3.23016718e-02,  4.63906303e-02,
         2.58311220e-02,  1.76636688e-02,  3.31863835e-02,
        -2.47860118e-03, -4.25862223e-02,  2.99789384e-02,
         2.67586531e-03,  1.81598496e-02,  1.37071135e-02,
        -3.10147200e-02, -3.98573913e-02,  2.17524301e-0

Then, we would either need to run this over all the dataframes rows, which is not wrong, but will take some time that we need ;)

Therefore we are going to create the embeddings in a separate file, associated with a title. Since there are onlt 266 products, we dont need more than that. But now is the part where we make our first decision. Either we do some feature engineering for creating a new df which contains unique visitID clicks, append all the products clicked in just one big string, and then perform the embeddings process, or, we can create the embeddings for the 266 products and then create some average. The first process seems more accurate but will take longer, and the second one is going to take less but could be less straightforward. 

We are going then to take both approaches to see where both go.


### First Approach - Concatenate Likes and Embed the  "Like List"

In [10]:
# Group by 'visitId' and aggregate 'Product name' using text join
grouped_df = (
    df.groupby("visitId")["ProductName"].apply(lambda x: ", ".join(x)).reset_index()
)

# Rename the aggregated column
grouped_df.rename(columns={"ProductName": "Clicks"}, inplace=True)

print(grouped_df)

         visitId                                             Clicks
0     1470035161                                  Google Power Bank
1     1470035457                         Deluge Waterproof Backpack
2     1470035521  Seat Pack Organizer, Seat Pack Organizer, Seat...
3     1470035892  Google Men's 100% Cotton Short Sleeve Hero Tee...
4     1470036291  Google Men's 100% Cotton Short Sleeve Hero Tee...
...          ...                                                ...
4630  1470896943  Google Toddler Tee Fruit Games Pineapple, Goog...
4631  1470896966  Google Canvas Tote Natural/Navy, Mistral Rucks...
4632  1470897377      Google Women's Lightweight Microfleece Jacket
4633  1470897905  Google Men's 100% Cotton Short Sleeve Hero Tee...
4634  1470897937          Google Women's Performance Golf Polo Blue

[4635 rows x 2 columns]


In [ ]:
from tqdm.notebook import tqdm_notebook

tqdm_notebook.pandas()  # fancy progress Bar

grouped_df["Embedings"] = grouped_df.Clicks.progress_apply(
    lambda x: model.create_embeddings(x)
)  # Make the embeddings for the Clicks column

In [54]:
grouped_df.Embeddings  # The embeddings

0       [[0.021230968, 0.06723951, -0.03646528, -0.005...
1       [[0.021318657, 0.04240716, -0.007994821, 0.004...
2       [[0.002186621, 0.051950835, -0.004776678, 0.02...
3       [[0.00616312, 0.04824848, -0.035558887, 0.0190...
4       [[-9.8550074e-05, 0.034511797, -0.016930008, 0...
                              ...                        
4630    [[0.0056050452, 0.029619096, -0.017937213, 0.0...
4631    [[0.02744697, 0.05886887, -0.022487251, 0.0082...
4632    [[0.010117159, 0.042951882, -0.020193057, -0.0...
4633    [[0.0072451867, 0.036614206, -0.039741036, 0.0...
4634    [[0.02905081, 0.046681486, -0.016705029, 0.021...
Name: Embeddings, Length: 4635, dtype: object

### Second Approach - Create our Embeddings Master File 

In [ ]:
tqdm_notebook.pandas()

inv_df = pd.DataFrame(
    [clicks_df.ProductName.unique(), clicks_df["ProductName"].value_counts()]
).T  # Products that have being clicked
inv_df.rename({0: "Product", 1: "Count"}, axis=1, inplace=True)

inv_df["Embeddings"] = inv_df.Product.progress_apply(model.create_embeddings)

In [59]:
inv_df.Embeddings

0      [[0.014249629, 0.023254616, -0.006114072, 0.00...
1      [[0.03786302, 0.03239363, -0.00020421806, 0.02...
2      [[0.00114997, 0.03650656, -0.035562657, 0.0202...
3      [[0.028030083, 0.038984273, -0.02813911, 0.034...
4      [[0.015068452, 0.05567977, -0.016779918, 0.011...
                             ...                        
261    [[0.021007422, 0.04520183, -0.023061385, 0.041...
262    [[0.013173798, 0.00889726, -0.012541152, 0.015...
263    [[0.008392215, 0.055684857, -0.011121839, -0.0...
264    [[-0.01466453, 0.04144204, -0.04656081, 0.0284...
265    [[0.0017286192, 0.04235564, -0.026112005, 0.01...
Name: Embeddings, Length: 266, dtype: object

In [17]:
## Now use this inv_df and original df to make a new df containing the embeddings.

clicks_df.rename({"ProductName": "Product"}, axis=1, inplace=1)

clicks_df = pd.merge(
    inv_df[["Product", "Embeddings"]], clicks_df, on="Product", how="right")

clicks_df

/var/folders/kr/9nlwnrjj3fl33l2k5m7w4snh0000gp/T/ipykernel_70202/589763138.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clicks_df.rename({'ProductName':'Product'},axis=1,inplace=1)


,Product,Embeddings,visitNumber,visitId,visitStartTime,date,total_visits,total_hits,total_pageviews,total_timeOnSite,...,Category,Variant,ProductList,transactionId,Revenue,itemTransactionId,Click,TotalPaid,Price,Quantity
0,Oasis Backpack,"[[0.014249629, 0.023254616, -0.006114072, 0.00...",2,1470857552,1470857552,20160810,1,5,4,125.0,...,Home/Limited Supply/Bags/,(not set),Category,NaN,0.0,NaN,True,0.0,99.99,NaN
1,Set of 3 Nested Travel Cases,"[[0.03786302, 0.03239363, -0.00020421806, 0.02...",1,1470862136,1470862136,20160810,1,5,4,341.0,...,Home/Limited Supply/Bags/,(not set),Category,NaN,0.0,NaN,True,0.0,8.79,NaN
2,YouTube Men's Short Sleeve Hero Tee Black,"[[0.00114997, 0.03650656, -0.035562657, 0.0202...",1,1470848713,1470848713,20160810,1,6,5,106.0,...,Home/Apparel/Men's/Men's-T-Shirts/,(not set),Category,NaN,0.0,NaN,True,0.0,16.99,NaN
3,Maze Pen,"[[0.028030083, 0.038984273, -0.02813911, 0.034...",4,1470815734,1470815734,20160810,1,6,3,8.0,...,Home/Office/,(not set),Category,NaN,0.0,NaN,True,0.0,0.99,NaN
4,Maze Pen,"[[0.028030083, 0.038984273, -0.02813911, 0.034...",4,1470815734,1470815734,20160810,1,6,3,8.0,...,Home/Office/,(not set),Category,NaN,0.0,NaN,True,0.0,0.99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20873,Google Women's Hero V-Neck Tee White,"[[0.018843824, 0.053733025, -0.02561408, 0.008...",7,1470120798,1470120798,20160801,1,34,18,300.0,...,Home/Apparel/Women's/Women's-T-Shirts/,(not set),(not set),NaN,0.0,NaN,True,0.0,16.99,NaN
20874,Google Women's Hero V-Neck Tee White,"[[0.018843824, 0.053733025, -0.02561408, 0.008...",7,1470120798,1470120798,20160801,1,34,18,300.0,...,Home/Apparel/Women's/Women's-T-Shirts/,(not set),(not set),NaN,0.0,NaN,True,0.0,16.99,NaN
20875,Google Women's Short Sleeve Hero Tee White,"[[-5.1616345e-05, 0.047178693, -0.030958962, 0...",7,1470120798,1470120798,20160801,1,34,18,300.0,...,Home/Apparel/Women's/Women's-T-Shirts/,(not set),(not set),NaN,0.0,NaN,True,0.0,17.99,NaN
20876,Google Women's Short Sleeve V-Neck Tee Lavender,"[[0.01456775, 0.050580125, -0.034778133, 0.027...",7,1470120798,1470120798,20160801,1,34,18,300.0,...,Home/Apparel/Women's/Women's-T-Shirts/,(not set),(not set),NaN,0.0,NaN,True,0.0,16.99,NaN


In [ ]:
##  Now we store pur embeddings so that we dont do the same all over again

import os

inv_df.to_hdf("google data/embeddings/inventory_embeddings.h5", key="data", mode="w")
grouped_df.to_hdf("google data/embeddings/likes_embeddings.h5", key="data", mode="w")

-------
## Second phase of the walkthrough here with the embeddings completed.


We can start directly her eif the embeddings is not in what we are interested in.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("google data/data.csv")
clicks_df = df.loc[df.Click == True]  # Original df

inv_df = pd.read_hdf(
    "google data/embeddings/inventory_embeddings.h5", key="data")  # Embeddings
likes_df = pd.read_hdf(
    "google data/embeddings/likes_embeddings.h5", key="data")  # Embeddings

clicks_df.rename({"ProductName": "Product"}, axis=1, inplace=1)
clicks_df = pd.merge(
    inv_df[["Product", "Embeddings"]], clicks_df, on="Product", how="right")

# Superior DF Creation

df = clicks_df.groupby("visitId")["Embeddings"].apply(
    np.mean)  # Apply a mean to embeddings across products clicked on the same session
df = df.reset_index()
df.columns = ["visitId", "Embeddings"]

features_list = [
    "visitId",
    "visitNumber",
    "total_visits",
    "total_hits",
    "Price",
    "total_timeOnSite",
    "total_newVisits",
    "total_pageviews",
]

df = pd.merge(df, clicks_df.loc[:, features_list], on="visitId", how="left")
df.drop_duplicates(subset="visitId", inplace=True)
df.reset_index(drop=True, inplace=True)

In [3]:
df.head()

,visitId,Embeddings,visitNumber,total_visits,total_hits,Price,total_timeOnSite,total_newVisits,total_pageviews
0,1470035161,"[[0.021230968, 0.06723951, -0.03646528, -0.005...",1,1,9,16.99,89.0,1.0,7
1,1470035457,"[[0.021318657, 0.04240716, -0.007994821, 0.004...",3,1,7,99.99,421.0,NaN,5
2,1470035521,"[[0.022162948, 0.049151335, -0.013284831, 0.02...",1,1,53,11.99,755.0,1.0,36
3,1470035892,"[[0.00616312, 0.04824848, -0.035558887, 0.0190...",1,1,8,16.99,82.0,1.0,6
4,1470036291,"[[0.0071259676, 0.043269128, -0.0370314, 0.012...",7,1,33,16.99,465.0,NaN,20


Now we need to implement something that creates a scoring based on the behavioral features. This way, the recommendation process will have more sense. In this use case, we are going to use the original to extract a scoring function using a "sell" as the target variable that we want to predict. This is the general objective of the e-commerce use case. You may think of how are we going to create a score based on this. 

The answer comes as a trick in transforming the data. We are going to take one set of data that contains visit ID who end up in purchases and cases in which are not. Then using a classic sigmoid head at the end of whatever our model is going to be, we end up getting the probabilities of one visit ID ending up in a purchase, which turns this also into a binary classification problem. Our score is going to be the probability that ends up in a selling.

In [10]:
data = pd.read_csv("google data/data.csv")
data

,visitNumber,visitId,visitStartTime,date,total_visits,total_hits,total_pageviews,total_timeOnSite,total_bounces,total_newVisits,...,Category,Variant,ProductList,transactionId,Revenue,itemTransactionId,Click,TotalPaid,Price,Quantity
0,2,1470857552,1470857552,20160810,1,5,4,125.0,NaN,NaN,...,Home/Limited Supply/Bags/,(not set),Category,NaN,0.0,NaN,True,0.0,99.99,NaN
1,1,1470862136,1470862136,20160810,1,5,4,341.0,NaN,1.0,...,Home/Limited Supply/Bags/,(not set),Category,NaN,0.0,NaN,True,0.0,8.79,NaN
2,1,1470848713,1470848713,20160810,1,6,5,106.0,NaN,1.0,...,Home/Apparel/Men's/Men's-T-Shirts/,(not set),Category,NaN,0.0,NaN,True,0.0,16.99,NaN
3,4,1470815734,1470815734,20160810,1,6,3,8.0,NaN,NaN,...,Home/Office/,(not set),Category,NaN,0.0,NaN,True,0.0,0.99,NaN
4,4,1470815734,1470815734,20160810,1,6,3,8.0,NaN,NaN,...,Home/Office/,(not set),Category,NaN,0.0,NaN,True,0.0,0.99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22817,7,1470120798,1470120798,20160801,1,34,18,300.0,NaN,NaN,...,Home/Apparel/Women's/Women's-T-Shirts/,(not set),(not set),NaN,0.0,NaN,True,0.0,16.99,NaN
22818,7,1470120798,1470120798,20160801,1,34,18,300.0,NaN,NaN,...,Home/Apparel/Women's/Women's-T-Shirts/,(not set),(not set),NaN,0.0,NaN,True,0.0,16.99,NaN
22819,7,1470120798,1470120798,20160801,1,34,18,300.0,NaN,NaN,...,Home/Apparel/Women's/Women's-T-Shirts/,(not set),(not set),NaN,0.0,NaN,True,0.0,17.99,NaN
22820,7,1470120798,1470120798,20160801,1,34,18,300.0,NaN,NaN,...,Home/Apparel/Women's/Women's-T-Shirts/,(not set),(not set),NaN,0.0,NaN,True,0.0,16.99,NaN


In [16]:
data_y

,visitId,Revenue
0,1470035161,0.0
1,1470035457,0.0
2,1470035521,0.0
3,1470035892,0.0
4,1470036291,0.0
...,...,...
4630,1470896943,0.0
4631,1470896966,0.0
4632,1470897377,0.0
4633,1470897905,0.0


In [9]:
data = pd.read_csv("google data/data.csv") # read data with both purchases and viewed articles mixed.

features_list = [
    "visitId",
    "visitNumber",
    "total_visits",
    "total_hits",
    "Price",
    "total_timeOnSite",
    "total_newVisits",
    "total_pageviews",
]

# Group by 'visitId' and aggregate 'Product name' using text join
data_x = data.groupby("visitId")["ProductName"].apply(lambda x: ", ".join(x)).reset_index()

# Group by visitid and aggregate 'Revenue'

data_y = data.groupby("visitId")["Revenue"].apply(np.sum).reset_index()

# Rename the aggregated column
data_x.rename(columns={"ProductName": "Clicks"}, inplace=True)

data = pd.merge(data_x,data.loc[:,features_list],on='visitId',how='left').drop_duplicates(subset='visitId').reset_index(drop=True)
data = pd.merge(data_y,)
data

,visitId,Clicks,visitNumber,total_visits,total_hits,Price,total_timeOnSite,total_newVisits,total_pageviews
0,1470035161,Google Power Bank,1,1,9,16.99,89.0,1.0,7
1,1470035457,Deluge Waterproof Backpack,3,1,7,99.99,421.0,NaN,5
2,1470035521,"Seat Pack Organizer, Seat Pack Organizer, Seat...",1,1,53,11.99,755.0,1.0,36
3,1470035892,Google Men's 100% Cotton Short Sleeve Hero Tee...,1,1,8,16.99,82.0,1.0,6
4,1470036291,Google Men's 100% Cotton Short Sleeve Hero Tee...,7,1,33,16.99,465.0,NaN,20
...,...,...,...,...,...,...,...,...,...
4630,1470896943,"Google Toddler Tee Fruit Games Pineapple, Goog...",1,1,15,23.99,178.0,1.0,7
4631,1470896966,"Google Canvas Tote Natural/Navy, Mistral Rucks...",1,1,12,15.99,240.0,1.0,9
4632,1470897377,Google Women's Lightweight Microfleece Jacket,2,1,3,74.99,25.0,NaN,2
4633,1470897905,Google Men's 100% Cotton Short Sleeve Hero Tee...,2,1,13,16.99,249.0,NaN,12
